In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


functions

In [2]:
def load_experiment_data(directory_path):
    # Define the names of the data folders and the label folder
    data_folders = ["Accelerometer_Data", "Force_Data", "Acoustic_Emission_Data"]
    label_folder = "Tool_wear_values"

    # Initialize a dictionary to store dataframes for each experiment
    experiment_data = {}

    # Loop through each experiment number
    for i in range(1, 13):
        experiment_name = f"Expt_{i}"
        experiment_data[experiment_name] = {}

        # Loop through each data folder and load the corresponding CSV
        for folder in data_folders:
            csv_path = os.path.join(directory_path, folder, f"{experiment_name}.csv")
            df = pd.read_csv(csv_path)
            experiment_data[experiment_name][folder] = df

    return experiment_data

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

class FeatureExtractor:
    def __init__(self, normal_ratio=0.5, abnormal_ratio=0.5):
        assert normal_ratio + abnormal_ratio == 1, "The sum of normal_ratio and abnormal_ratio must be 1."
        self.normal_ratio = normal_ratio
        self.abnormal_ratio = abnormal_ratio

    def calculate_features(self, data):
        Z1 = np.mean(np.abs(data))  #  (Mean Value)
        Z2 = np.sqrt(np.mean(data ** 2))  # RMS
        Z3 = np.std(data)  # Standard Deviation
        Z4 = Z2 / Z1  # Shape Factor
        Z5 = np.mean(((np.abs(data - Z1)) / Z3) ** 3)  # Skewness
        Z6 = np.mean(((np.abs(data - Z1)) / Z3) ** 4)  # Kurtosis
        Z7 = np.max(np.abs(data))  # Peak Value
        Z8 = Z7 / Z2  # Crest Factor
        Z9 = Z7 / Z1  # Impulse Factor
        Z10 = np.sum([(f ** 2) * p for f, p in enumerate(np.abs(np.fft.fft(data)) ** 2)])  #  (MSF)
        Z11 = np.mean(np.abs(np.fft.fft(data)) ** 2)  # (MPS)
        Z12 = np.sum([f * p for f, p in enumerate(np.abs(np.fft.fft(data)) ** 2)]) / np.sum(np.abs(np.fft.fft(data)) ** 2)  # (FC)
        return [Z1, Z2, Z3, Z4, Z5, Z6, Z7, Z8, Z9, Z10, Z11, Z12]

    def extract_features(self, expt_data):
        normal_features = []
        abnormal_features = []

        for key, df in expt_data.items():  # (Accelerometer, Force, Acoustic)
            n_normal = int(len(df) * self.normal_ratio)
            n_abnormal = len(df) - n_normal

            normal_data = df.iloc[:n_normal]
            abnormal_data = df.iloc[-n_abnormal:]

            for col in df.columns:
                normal_features.append(self.calculate_features(normal_data[col]))
                abnormal_features.append(self.calculate_features(abnormal_data[col]))

        normal_features = np.array(normal_features).flatten()
        abnormal_features = np.array(abnormal_features).flatten()

        return normal_features, abnormal_features

    def t_test_features(self, normal_features, abnormal_features):
        t_stat, p_value = ttest_ind(normal_features, abnormal_features, equal_var=False)
        return {'t_stat': t_stat, 'p_value': p_value}

    def process_experiment(self, experiment_data):
        feature_results = {}
        t_test_results = {}

        for expt_name, expt_data in experiment_data.items():
            normal_features, abnormal_features = self.extract_features(expt_data)
            feature_results[expt_name] = {'normal': normal_features, 'abnormal': abnormal_features}
            t_test_results[expt_name] = self.t_test_features(normal_features, abnormal_features)

        return feature_results, t_test_results




In [5]:
import numpy as np

# find normal_ratio & abnormal_ratio
def find_optimal_ratio(experiment_data, ratios):
    optimal_ratio = None
    min_avg_p_value = float('inf')
    p_values_dict = {}

    for normal_ratio in ratios:
        abnormal_ratio = 1 - normal_ratio
        extractor = FeatureExtractor(normal_ratio=normal_ratio, abnormal_ratio=abnormal_ratio)
        feature_results, t_test_results = extractor.process_experiment(experiment_data)


        avg_p_value = np.mean([result['p_value'] for result in t_test_results.values()])
        p_values_dict[normal_ratio] = avg_p_value


        if avg_p_value < min_avg_p_value:
            min_avg_p_value = avg_p_value
            optimal_ratio = normal_ratio

    return optimal_ratio, min_avg_p_value, p_values_dict



데이터 확인

In [6]:
#T1~T8 까지
import os
import pandas as pd

directories = [f'/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T{i}' for i in range(1, 9)]


for i, directory_path in enumerate(directories, 1):
    print(f"Processing T{i}...")
    experiment_data = load_experiment_data(directory_path)

    # 변수 길이 출력
    for expt_name, data in experiment_data.items():
        print(f"{expt_name} Variable Lengths:")
        for var_name, var_data in data.items():
            print(f"  {var_name}: {len(var_data)}")

    print("\n" + "="*50 + "\n")


Processing T1...
Expt_1 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 3800
Expt_2 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 4400
Expt_3 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 6500
Expt_4 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 6700
Expt_5 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 6400
Expt_6 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 6200
Expt_7 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 6700
Expt_8 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 60001
  Acoustic_Emission_Data: 6900
Expt_9 Variable Lengths:
  Accelerometer_Data: 337501
  Force_Data: 54001
  Acoustic_Emission_Data: 6900
Expt_10 Variable Lengths:
  Accelerome

T-Test : Normal, Abnormal 구간 분리 비율 특정하기


In [9]:
#개별 데이터셋에 대한 Abnormal, Normal 구간 구분
directory_path = '/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T1'  # 실제 T1 디렉토리 경로로 변경
experiment_data = load_experiment_data(directory_path)
ratios = np.arange(0.5, 1.0, 0.1)  # 0.5부터 0.9까지의 비율
all_experiments = {f"Expt_{i}": experiment_data[f"Expt_{i}"] for i in range(1, 13)}

# 개별 실험에 대해 최적의 비율과 최소 p-value 계산 및 출력
for expt_name, data in all_experiments.items():
    optimal_ratio, min_avg_p_value, p_values_dict = find_optimal_ratio({expt_name: data}, ratios)
    print(f"Experiment: {expt_name}")
    print(f"Optimal Normal Ratio: {optimal_ratio}")
    print(f"Minimum Average p-value: {min_avg_p_value}")

    # 각 비율에 대한 평균 p-value 출력
    for ratio, avg_p_value in p_values_dict.items():
        print(f"  Normal Ratio: {ratio}, Average p-value: {avg_p_value}")
    print('-' * 50)


Experiment: Expt_1
Optimal Normal Ratio: 0.7999999999999999
Minimum Average p-value: 0.1434573213141535
  Normal Ratio: 0.5, Average p-value: 0.9242716070627832
  Normal Ratio: 0.6, Average p-value: 0.22934874203470584
  Normal Ratio: 0.7, Average p-value: 0.15098356628041573
  Normal Ratio: 0.7999999999999999, Average p-value: 0.1434573213141535
  Normal Ratio: 0.8999999999999999, Average p-value: 0.14381858982019308
--------------------------------------------------
Experiment: Expt_2
Optimal Normal Ratio: 0.7999999999999999
Minimum Average p-value: 0.10539333332058415
  Normal Ratio: 0.5, Average p-value: 0.6339741599051424
  Normal Ratio: 0.6, Average p-value: 0.2541139618070033
  Normal Ratio: 0.7, Average p-value: 0.11879089042456481
  Normal Ratio: 0.7999999999999999, Average p-value: 0.10539333332058415
  Normal Ratio: 0.8999999999999999, Average p-value: 0.10729606661485076
--------------------------------------------------
Experiment: Expt_3
Optimal Normal Ratio: 0.7999999999

전체 T1~T8에 대한 구간 구분

In [10]:

directories = [f'/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T{i}' for i in range(1, 9)]

for i, directory_path in enumerate(directories, 1):
    print(f"Processing T{i}...")
    experiment_data = load_experiment_data(directory_path)

    ratios = np.arange(0.5, 1.0, 0.1)  # 0.5부터 0.9까지의 비율


    all_experiments = {f"Expt_{j}": experiment_data[f"Expt_{j}"] for j in range(1, 13)}
    optimal_ratio, min_avg_p_value, p_values_dict = find_optimal_ratio(all_experiments, ratios)


    print(f"T{i} Optimal Normal Ratio: {optimal_ratio}")
    print(f"T{i} Minimum Average p-value: {min_avg_p_value}")
    print(f"T{i} Average p-values per ratio:")

    for ratio, avg_p_value in p_values_dict.items():
        print(f"Normal Ratio: {ratio}, Average p-value: {avg_p_value}")

    print("\n" + "="*50 + "\n")


Processing T1...
T1 Optimal Normal Ratio: 0.7999999999999999
T1 Minimum Average p-value: 0.09532511840231866
T1 Average p-values per ratio:
Normal Ratio: 0.5, Average p-value: 0.7263697599157614
Normal Ratio: 0.6, Average p-value: 0.18721337383959302
Normal Ratio: 0.7, Average p-value: 0.10371643874201346
Normal Ratio: 0.7999999999999999, Average p-value: 0.09532511840231866
Normal Ratio: 0.8999999999999999, Average p-value: 0.09593891946678691


Processing T2...
T2 Optimal Normal Ratio: 0.8999999999999999
T2 Minimum Average p-value: 0.07267334580165906
T2 Average p-values per ratio:
Normal Ratio: 0.5, Average p-value: 0.5669823321157501
Normal Ratio: 0.6, Average p-value: 0.13747701637074702
Normal Ratio: 0.7, Average p-value: 0.08123321395205196
Normal Ratio: 0.7999999999999999, Average p-value: 0.0743772649415158
Normal Ratio: 0.8999999999999999, Average p-value: 0.07267334580165906


Processing T3...
T3 Optimal Normal Ratio: 0.7999999999999999
T3 Minimum Average p-value: 0.10299846

Prognostic 구간 특정

In [15]:
def load_experiment_data2(directory_path):
    # Define the names of the data folders and the label folder
    data_folders = ["Accelerometer_Data", "Force_Data", "Acoustic_Emission_Data"]
    label_folder = "Tool_wear_values"

    # Initialize a dictionary to store dataframes for each experiment
    experiment_data = {}

    # Loop through each experiment number
    for i in range(1, 13):
        experiment_name = f"Expt_{i}"
        experiment_data[experiment_name] = {}

        # Loop through each data folder and load the corresponding CSV
        for folder in data_folders:
            csv_path = os.path.join(directory_path, folder, f"{experiment_name}.csv")
            df = pd.read_csv(csv_path)

            # use only 80%
            df_80_percent = df.iloc[:int(len(df) * 0.8)]
            experiment_data[experiment_name][folder] = df_80_percent

    return experiment_data

In [16]:

directories = [f'/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T{i}' for i in range(1, 9)]

for i, directory_path in enumerate(directories, 1):
    print(f"Processing T{i}...")
    experiment_data = load_experiment_data2(directory_path)


    ratios = np.arange(0.5, 1.0, 0.1)


    all_experiments = {f"Expt_{j}": experiment_data[f"Expt_{j}"] for j in range(1, 13)}
    optimal_ratio, min_avg_p_value, p_values_dict = find_optimal_ratio(all_experiments, ratios)

    print(f"T{i} Optimal Normal Ratio: {optimal_ratio}")
    print(f"T{i} Minimum Average p-value: {min_avg_p_value}")
    print(f"T{i} Average p-values per ratio:")

    for ratio, avg_p_value in p_values_dict.items():
        print(f"Normal Ratio: {ratio}, Average p-value: {avg_p_value}")

    print("\n" + "="*50 + "\n")

Processing T1...
T1 Optimal Normal Ratio: 0.7999999999999999
T1 Minimum Average p-value: 0.09262491638089003
T1 Average p-values per ratio:
Normal Ratio: 0.5, Average p-value: 0.7624784561427623
Normal Ratio: 0.6, Average p-value: 0.1728913015603217
Normal Ratio: 0.7, Average p-value: 0.1000122061144601
Normal Ratio: 0.7999999999999999, Average p-value: 0.09262491638089003
Normal Ratio: 0.8999999999999999, Average p-value: 0.09295512028110409


Processing T2...
T2 Optimal Normal Ratio: 0.8999999999999999
T2 Minimum Average p-value: 0.07554263352332986
T2 Average p-values per ratio:
Normal Ratio: 0.5, Average p-value: 0.6343037744290286
Normal Ratio: 0.6, Average p-value: 0.18575894315336816
Normal Ratio: 0.7, Average p-value: 0.08531424800036282
Normal Ratio: 0.7999999999999999, Average p-value: 0.07709719045753627
Normal Ratio: 0.8999999999999999, Average p-value: 0.07554263352332986


Processing T3...
T3 Optimal Normal Ratio: 0.7999999999999999
T3 Minimum Average p-value: 0.098459307

레이블 연장 작업

In [30]:
import os
import numpy as np
import pandas as pd


directories = [f'/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T{i}' for i in range(1, 9)]


all_data = []

for i, directory in enumerate(directories, 1):
    csv_path = os.path.join(directory, "Tool_Wear_Values", f"T{i}_Tool_wear_values_for_all_Experiments.csv")

    try:
        df = pd.read_csv(csv_path, encoding='latin1')
    except UnicodeDecodeError:
        print(f"Error reading {csv_path}, trying a different encoding.")
        continue

    df['Settings'] = f"T{i}"

    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)


def generate_continuous_wear_values(end, num_points=1000):
    return np.linspace(0, end, num_points)


expanded_data = []

for i, (index, row) in enumerate(combined_df.iterrows()):
    exp = f"Expt_{(i % 12) + 1}"  # Expt_1 ~ Expt_12
    setting = row['Settings']
    wear_value = row['Tool Wear in (µm)']


    continuous_wear = generate_continuous_wear_values(wear_value)


    expanded_data.append([exp, setting] + list(continuous_wear))


expanded_df = pd.DataFrame(expanded_data)

columns = ['Expt', 'Settings'] + [f'Tool Wear in (µm)_{i}' for i in range(1, 1001)]
expanded_df.columns = columns



     Expt Settings  Tool Wear in (µm)_1  Tool Wear in (µm)_2  \
0  Expt_1       T1                  0.0             0.123472   
1  Expt_2       T1                  0.0             0.144370   
2  Expt_3       T1                  0.0             0.205787   
3  Expt_4       T1                  0.0             0.219678   
4  Expt_5       T1                  0.0             0.229041   

   Tool Wear in (µm)_3  Tool Wear in (µm)_4  Tool Wear in (µm)_5  \
0             0.246945             0.370417             0.493890   
1             0.288741             0.433111             0.577481   
2             0.411574             0.617360             0.823147   
3             0.439355             0.659033             0.878711   
4             0.458082             0.687123             0.916164   

   Tool Wear in (µm)_6  Tool Wear in (µm)_7  Tool Wear in (µm)_8  ...  \
0             0.617362             0.740835             0.864307  ...   
1             0.721852             0.866222             1.01

In [31]:
expanded_df

,Expt,Settings,Tool Wear in (µm)_1,Tool Wear in (µm)_2,Tool Wear in (µm)_3,Tool Wear in (µm)_4,Tool Wear in (µm)_5,Tool Wear in (µm)_6,Tool Wear in (µm)_7,Tool Wear in (µm)_8,...,Tool Wear in (µm)_991,Tool Wear in (µm)_992,Tool Wear in (µm)_993,Tool Wear in (µm)_994,Tool Wear in (µm)_995,Tool Wear in (µm)_996,Tool Wear in (µm)_997,Tool Wear in (µm)_998,Tool Wear in (µm)_999,Tool Wear in (µm)_1000
0,Expt_1,T1,0.0,0.123472,0.246945,0.370417,0.493890,0.617362,0.740835,0.864307,...,122.237748,122.361220,122.484693,122.608165,122.731638,122.855110,122.978583,123.102055,123.225528,123.349
1,Expt_2,T1,0.0,0.144370,0.288741,0.433111,0.577481,0.721852,0.866222,1.010593,...,142.926667,143.071037,143.215407,143.359778,143.504148,143.648519,143.792889,143.937259,144.081630,144.226
2,Expt_3,T1,0.0,0.205787,0.411574,0.617360,0.823147,1.028934,1.234721,1.440508,...,203.728919,203.934706,204.140492,204.346279,204.552066,204.757853,204.963640,205.169426,205.375213,205.581
3,Expt_4,T1,0.0,0.219678,0.439355,0.659033,0.878711,1.098388,1.318066,1.537744,...,217.480901,217.700579,217.920256,218.139934,218.359612,218.579289,218.798967,219.018645,219.238322,219.458
4,Expt_5,T1,0.0,0.229041,0.458082,0.687123,0.916164,1.145205,1.374246,1.603287,...,226.750631,226.979672,227.208713,227.437754,227.666795,227.895836,228.124877,228.353918,228.582959,228.812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Expt_8,T8,0.0,0.216977,0.433954,0.650931,0.867908,1.084885,1.301862,1.518839,...,214.807207,215.024184,215.241161,215.458138,215.675115,215.892092,216.109069,216.326046,216.543023,216.760
92,Expt_9,T8,0.0,0.228679,0.457357,0.686036,0.914715,1.143393,1.372072,1.600751,...,226.391892,226.620571,226.849249,227.077928,227.306607,227.535285,227.763964,227.992643,228.221321,228.450
93,Expt_10,T8,0.0,0.247688,0.495375,0.743063,0.990751,1.238438,1.486126,1.733814,...,245.210811,245.458498,245.706186,245.953874,246.201562,246.449249,246.696937,246.944625,247.192312,247.440
94,Expt_11,T8,0.0,0.283684,0.567367,0.851051,1.134735,1.418418,1.702102,1.985786,...,280.846847,281.130531,281.414214,281.697898,281.981582,282.265265,282.548949,282.832633,283.116316,283.400


In [39]:
import os
import numpy as np
import pandas as pd

directories = [f'/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T{i}' for i in range(1, 9)]


all_data = []


for i, directory in enumerate(directories, 1):
    csv_path = os.path.join(directory, "Tool_Wear_Values", f"T{i}_Tool_wear_values_for_all_Experiments.csv")

    try:
        df = pd.read_csv(csv_path, encoding='latin1')
    except UnicodeDecodeError:
        print(f"Error reading {csv_path}, trying a different encoding.")
        continue

    df['Settings'] = f"T{i}"

    all_data.append(df)


combined_df = pd.concat(all_data, ignore_index=True)


def generate_log_wear_values(end, num_points=1000):
    return np.log1p(np.linspace(0, end, num_points))


expanded_data = []

for i, (index, row) in enumerate(combined_df.iterrows()):
    exp = f"Expt_{(i % 12) + 1}"  # Expt_1 ~ Expt_12
    setting = row['Settings']
    wear_value = row['Tool Wear in (µm)']


    continuous_wear = generate_log_wear_values(wear_value)

    expanded_data.append([exp, setting] + list(continuous_wear))

expanded_df_log = pd.DataFrame(expanded_data)


columns = ['Expt', 'Settings'] + [f'Log Tool Wear in (µm)_{i}' for i in range(1, 1001)]
expanded_df_log.columns = columns




In [40]:
expanded_df_log

,Expt,Settings,Log Tool Wear in (µm)_1,Log Tool Wear in (µm)_2,Log Tool Wear in (µm)_3,Log Tool Wear in (µm)_4,Log Tool Wear in (µm)_5,Log Tool Wear in (µm)_6,Log Tool Wear in (µm)_7,Log Tool Wear in (µm)_8,...,Log Tool Wear in (µm)_991,Log Tool Wear in (µm)_992,Log Tool Wear in (µm)_993,Log Tool Wear in (µm)_994,Log Tool Wear in (µm)_995,Log Tool Wear in (µm)_996,Log Tool Wear in (µm)_997,Log Tool Wear in (µm)_998,Log Tool Wear in (µm)_999,Log Tool Wear in (µm)_1000
0,Expt_1,T1,0.0,0.116424,0.220697,0.315115,0.401383,0.480797,0.554365,0.622890,...,4.814115,4.815117,4.816117,4.817117,4.818115,4.819112,4.820109,4.821104,4.822099,4.823092
1,Expt_2,T1,0.0,0.134855,0.253666,0.359848,0.455830,0.543400,0.623916,0.698430,...,4.969304,4.970306,4.971308,4.972309,4.973308,4.974307,4.975304,4.976301,4.977297,4.978291
2,Expt_3,T1,0.0,0.187132,0.344705,0.480795,0.600564,0.707510,0.804116,0.892206,...,5.321687,5.322691,5.323695,5.324698,5.325699,5.326700,5.327700,5.328698,5.329696,5.330693
3,Expt_4,T1,0.0,0.198587,0.364195,0.506235,0.630586,0.741170,0.840733,0.931275,...,5.386699,5.387704,5.388708,5.389710,5.390712,5.391713,5.392713,5.393712,5.394710,5.395707
4,Expt_5,T1,0.0,0.206234,0.377122,0.523025,0.650325,0.763235,0.864680,0.956775,...,5.428251,5.429256,5.430261,5.431264,5.432266,5.433267,5.434267,5.435266,5.436264,5.437262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Expt_8,T8,0.0,0.196370,0.360436,0.501339,0.624819,0.734714,0.833718,0.923798,...,5.374385,5.375390,5.376394,5.377397,5.378399,5.379400,5.380400,5.381399,5.382397,5.383394
92,Expt_9,T8,0.0,0.205939,0.376625,0.522380,0.649569,0.762390,0.863764,0.955800,...,5.426675,5.427680,5.428684,5.429687,5.430689,5.431691,5.432691,5.433690,5.434688,5.435685
93,Expt_10,T8,0.0,0.221292,0.402377,0.555644,0.688512,0.805778,0.910726,1.005698,...,5.506188,5.507194,5.508198,5.509202,5.510204,5.511206,5.512206,5.513205,5.514204,5.515201
94,Expt_11,T8,0.0,0.249734,0.449397,0.615754,0.758342,0.883114,0.994030,1.093863,...,5.641364,5.642370,5.643375,5.644379,5.645382,5.646384,5.647385,5.648385,5.649384,5.650382


In [42]:
import os
import numpy as np
import pandas as pd


directories = [f'/content/drive/MyDrive/COOP/0814_은수님작업연장/Dataset/T{i}' for i in range(1, 9)]

all_data = []

for i, directory in enumerate(directories, 1):
    csv_path = os.path.join(directory, "Tool_Wear_Values", f"T{i}_Tool_wear_values_for_all_Experiments.csv")


    try:
        df = pd.read_csv(csv_path, encoding='latin1')
    except UnicodeDecodeError:
        print(f"Error reading {csv_path}, trying a different encoding.")
        continue

    df['Settings'] = f"T{i}"


    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

def generate_log_wear_values(end, num_points=1000):
    return np.log1p(np.linspace(0, end, num_points))


def generate_labels(num_points=1000):
    normal_count = int(num_points * 16 / (16 + 4 + 5))
    prognostic_count = int(num_points * 4 / (16 + 4 + 5))
    abnormal_count = num_points - normal_count - prognostic_count
    labels = ['Normal'] * normal_count + ['Prognostic'] * prognostic_count + ['Abnormal'] * abnormal_count
    return labels

expanded_data = []
label_data = []

for i, (index, row) in enumerate(combined_df.iterrows()):
    exp = f"Expt_{(i % 12) + 1}"  # Expt_1 ~ Expt_12
    setting = row['Settings']
    wear_value = row['Tool Wear in (µm)']


    continuous_wear = generate_log_wear_values(wear_value)


    expanded_data.append([exp, setting] + list(continuous_wear))


    labels = generate_labels(len(continuous_wear))
    label_data.append([exp, setting] + labels)


expanded_df_log = pd.DataFrame(expanded_data)


label_df = pd.DataFrame(label_data)


columns = ['Expt', 'Settings'] + [f'label for_{i}th toolwear' for i in range(1, 1001)]
expanded_df_log.columns = columns

label_columns = ['Expt', 'Settings'] + [f'Label_{i}' for i in range(1, 1001)]
label_df.columns = label_columns


print(label_df.head())


     Expt Settings Label_1 Label_2 Label_3 Label_4 Label_5 Label_6 Label_7  \
0  Expt_1       T1  Normal  Normal  Normal  Normal  Normal  Normal  Normal   
1  Expt_2       T1  Normal  Normal  Normal  Normal  Normal  Normal  Normal   
2  Expt_3       T1  Normal  Normal  Normal  Normal  Normal  Normal  Normal   
3  Expt_4       T1  Normal  Normal  Normal  Normal  Normal  Normal  Normal   
4  Expt_5       T1  Normal  Normal  Normal  Normal  Normal  Normal  Normal   

  Label_8  ... Label_991 Label_992 Label_993 Label_994 Label_995 Label_996  \
0  Normal  ...  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal   
1  Normal  ...  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal   
2  Normal  ...  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal   
3  Normal  ...  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal   
4  Normal  ...  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal  Abnormal   

  Label_997 Label_998 Label_999 Label_1000  
0  Abnormal  Abno

In [43]:
label_df

,Expt,Settings,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,...,Label_991,Label_992,Label_993,Label_994,Label_995,Label_996,Label_997,Label_998,Label_999,Label_1000
0,Expt_1,T1,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
1,Expt_2,T1,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
2,Expt_3,T1,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
3,Expt_4,T1,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
4,Expt_5,T1,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Expt_8,T8,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
92,Expt_9,T8,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
93,Expt_10,T8,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal
94,Expt_11,T8,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,...,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal,Abnormal


In [44]:
label_df.to_csv('/content/drive/MyDrive/COOP/0814_은수님작업연장/label_toolwear.csv', index=False)
